In [261]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
import mlflow
import mlflow.sklearn
import mlflow.xgboost
df= pd.read_csv('GUIDE_Train.csv',nrows=100000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [262]:
df.columns = df.columns.str.lower()
df = df.drop(columns=['id'])

In [263]:
import pandas as pd
import numpy as np
from sklearn.utils import resample

# Assuming your dataframe is called 'df' and the column is 'incidentgrade'

# Get the value counts
value_counts = df['incidentgrade'].value_counts()

# Find the minimum count
min_count = value_counts.min()

# Create a list to store the balanced dataframes
balanced_dfs = []

# Undersample each class
for class_value in value_counts.index:
    class_df = df[df['incidentgrade'] == class_value]

    if len(class_df) > min_count:
        # Undersample
        undersampled_df = resample(class_df,
                                   replace=False,    # sample without replacement
                                   n_samples=min_count,
                                   random_state=42)  # reproducible results
        balanced_dfs.append(undersampled_df)
    else:
        # If this class is already at or below the minimum, keep all samples
        balanced_dfs.append(class_df)

# Combine the balanced dataframes
df_balanced = pd.concat(balanced_dfs)

# Verify the new class distribution
print(df_balanced['incidentgrade'].value_counts())

df = df_balanced

incidentgrade
BenignPositive    21564
TruePositive      21564
FalsePositive     21564
Name: count, dtype: int64


In [264]:
missing_percentage = (df.isna().sum() / len(df)) * 100

missing_percentage = missing_percentage.sort_values(ascending=False)

missing_percentage.head(12)

actiongranular       99.931985
actiongrouped        99.931985
resourcetype         99.918073
threatfamily         99.261114
emailclusterid       99.024609
antispamdirection    98.347555
roles                97.596302
suspicionlevel       85.968899
lastverdict          78.062202
mitretechniques      55.863167
state                 0.000000
registryvaluename     0.000000
dtype: float64

In [265]:
column=['resourcetype',
        'actiongrouped',
        'actiongranular',
        'threatfamily',
        'emailclusterid',
        'antispamdirection',
        'roles',
        'suspicionlevel',
        'lastverdict',
        'mitretechniques']

df = df.drop(columns=column)

In [266]:

df['timestamp'] = pd.to_datetime(df['timestamp'])
# df['timestamp'] = pd.to_datetime(df['timestamp'])
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day
df['hour'] = df['timestamp'].dt.hour
df['minute'] = df['timestamp'].dt.minute

# Drop the original timestamp column
df.drop('timestamp', axis=1, inplace=True)

In [267]:
value_count = {col: df[col].nunique() for col in df.columns}

value_count_df = pd.DataFrame(list(value_count.items()), columns=['Column', 'Unique Values'])
value_count_df_sorted = value_count_df.sort_values(by='Unique Values',ascending=False)

value_count_df_sorted

,Column,Unique Values
2,alertid,54274
1,incidentid,35061
14,accountupn,18761
16,accountname,12830
13,accountsid,12306
15,accountobjectid,12150
11,ipaddress,9934
4,alerttitle,8362
18,networkmessageid,7054
25,filename,3907


In [213]:
# high_cardinolity_column = ['alertid','incidentid','accountupn','accountname','accountsid','accountobjectid','ipaddress']
# df = df.drop(columns=high_cardinolity_column)

In [268]:
df.head(2)

,orgid,incidentid,alertid,detectorid,alerttitle,category,incidentgrade,entitytype,evidencerole,deviceid,sha256,ipaddress,url,accountsid,accountupn,accountobjectid,accountname,devicename,networkmessageid,registrykey,registryvaluename,registryvaluedata,applicationid,applicationname,oauthapplicationid,filename,folderpath,resourceidname,osfamily,osversion,countrycode,state,city,month,day,hour,minute
84067,69,151193,1544727,1546,2028,Execution,BenignPositive,Process,Related,98799,138268,360606,160396,441377,673934,425863,453297,153085,529644,1631,635,860,2251,3421,881,289573,117668,3586,5,66,242,1445,10630,6,4,22,38
98788,24,220296,60883,13,11,InitialAccess,BenignPositive,User,Impacted,98799,138268,360606,160396,70309,45580,79277,69916,153085,529644,1631,635,860,2251,3421,881,289573,117668,3586,5,66,242,1445,10630,6,16,1,2


In [269]:
df = df.astype('category')

### Algoritham based imputation

In [216]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import LabelEncoder

# def custom_model_imputation(df, target_column):
#     # Encode categorical variables using LabelEncoder
#     label_encoders = {}
    
#     # Apply label encoding to all categorical columns
#     for col in df.select_dtypes(include=['object']).columns:
#         le = LabelEncoder()
#         df[col] = df[col].astype(str)  # Convert to string type if necessary
#         df[col] = le.fit_transform(df[col])
#         label_encoders[col] = le  # Store the encoder to reverse the encoding later if needed

#     # Encode the target column if it's categorical
#     if df[target_column].dtype == 'object' or df[target_column].dtype.name == 'category':
#         le_target = LabelEncoder()
#         df[target_column] = df[target_column].astype(str)
#         df[target_column] = le_target.fit_transform(df[target_column])
#         label_encoders[target_column] = le_target  # Store the target encoder
    
#     # Check if there are missing values in the target column
#     if df[target_column].isnull().sum() == 0:
#         print(f"No missing values found in {target_column}, skipping imputation.")
#         return df, label_encoders

#     # Split the data into training (non-missing) and data with missing values
#     train_data = df.dropna(subset=[target_column])
#     missing_data = df[df[target_column].isnull()]

#     # Separate features and target
#     X_train = train_data.drop(columns=[target_column])
#     y_train = train_data[target_column]

#     # Train a Random Forest Classifier
#     rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
#     rf_model.fit(X_train, y_train)

#     # Predict missing values only if there are missing samples
#     X_missing = missing_data.drop(columns=[target_column])
#     if X_missing.shape[0] > 0:
#         predicted_values = rf_model.predict(X_missing)

#         # Fill missing values in the original dataframe
#         df.loc[df[target_column].isnull(), target_column] = predicted_values
#     else:
#         print(f"No missing rows to predict in {target_column}")

#     return df, label_encoders

# # Example of using the function on categorical columns
# df, label_encoders = custom_model_imputation(df, 'suspicionlevel')
# df, label_encoders = custom_model_imputation(df, 'lastverdict')
# df, label_encoders = custom_model_imputation(df, 'mitretechniques')


### Chi-Square Test

In [271]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Encode categorical features
le = LabelEncoder()
for col in df.select_dtypes(include='category').columns:
    df[col] = le.fit_transform(df[col])
# Define features and target
X = df.drop(columns=['incidentgrade'])  # Replace 'target' with your actual target column
y = df['incidentgrade']


# Apply Chi-Square Test
chi2_values, p_values = chi2(X, y)

# Create DataFrame for better readability
chi2_df = pd.DataFrame({
    'Feature': X.columns,
    'Chi2 Score': chi2_values
}).sort_values(by='Chi2 Score', ascending=False)

print(chi2_df)


               Feature    Chi2 Score
1           incidentid  5.342769e+07
2              alertid  2.223124e+07
4           alerttitle  1.095218e+07
10           ipaddress  2.401792e+06
14     accountobjectid  1.735125e+06
12          accountsid  1.677845e+06
15         accountname  1.556729e+06
0                orgid  1.102872e+06
3           detectorid  9.276799e+05
13          accountupn  6.910054e+05
17    networkmessageid  3.891619e+05
24            filename  3.337569e+05
16          devicename  1.909553e+05
25          folderpath  1.881554e+05
31                city  1.812527e+05
9               sha256  1.399948e+05
30               state  7.778095e+04
11                 url  5.408838e+04
29         countrycode  3.094722e+04
8             deviceid  8.094400e+03
33                 day  4.941292e+03
5             category  3.472607e+03
6           entitytype  1.543264e+03
34                hour  8.060775e+02
7         evidencerole  6.692036e+02
22     applicationname  1.235431e+02
2

In [219]:
# column = ['applicationid','osversion','minute','registrykey','month','osfamily','resourceidname','registryvaluedata','registryvaluename','oauthapplicationid']
# df = df.drop(columns=column)

### feature importance

In [272]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Example: Your DataFrame 'df' with features and target column
X = df.drop(columns=['incidentgrade'])  # Replace 'target' with your actual target column
y = df['incidentgrade']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Get feature importance
feature_importances = rf_model.feature_importances_

# Create a DataFrame to display feature importance
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
})

# Sort the DataFrame by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
print(feature_importance_df)


               Feature  Importance
0                orgid    0.220165
1           incidentid    0.136110
2              alertid    0.104836
3           detectorid    0.101129
4           alerttitle    0.076813
33                 day    0.047209
5             category    0.036065
35              minute    0.035990
34                hour    0.035601
13          accountupn    0.024089
6           entitytype    0.022126
10           ipaddress    0.017610
12          accountsid    0.016359
14     accountobjectid    0.016311
15         accountname    0.016009
17    networkmessageid    0.013597
16          devicename    0.010093
29         countrycode    0.009797
24            filename    0.008374
11                 url    0.008129
9               sha256    0.006996
25          folderpath    0.006569
7         evidencerole    0.005486
31                city    0.005485
8             deviceid    0.004385
32               month    0.004147
30               state    0.003559
22     applicationna

In [221]:
# important_features = [
#     'orgid', 
#     'incidentid', 
#     'alertid', 
#     'detectorid', 
#     'alerttitle', 
#     'day', 
#     'category', 
#     'minute', 
#     'hour', 
#     'accountupn', 
#     'entitytype', 
#     'ipaddress', 
#     'accountsid', 
#     'accountobjectid', 
#     'accountname',
#     'incidentgrade'
# ]

# df = df[important_features]


In [273]:
df.columns

Index(['orgid', 'incidentid', 'alertid', 'detectorid', 'alerttitle',
       'category', 'incidentgrade', 'entitytype', 'evidencerole', 'deviceid',
       'sha256', 'ipaddress', 'url', 'accountsid', 'accountupn',
       'accountobjectid', 'accountname', 'devicename', 'networkmessageid',
       'registrykey', 'registryvaluename', 'registryvaluedata',
       'applicationid', 'applicationname', 'oauthapplicationid', 'filename',
       'folderpath', 'resourceidname', 'osfamily', 'osversion', 'countrycode',
       'state', 'city', 'month', 'day', 'hour', 'minute'],
      dtype='object')

In [223]:
# categorical_column = df.select_dtypes(include='object').columns

# le = LabelEncoder()
# for col in categorical_column:
#     df[col] = le.fit_transform(df[col])

In [274]:
X = df.drop(columns=['incidentgrade']) 
y = df['incidentgrade']

In [275]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)


### StandardScaler

In [226]:
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

In [276]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier()
model_rf.fit(X_train, y_train)
params = model_rf.get_params()

In [277]:
model_xgb = XGBClassifier()
model_xgb.fit(X_train,y_train)
params = model_xgb.get_params()

In [279]:
from sklearn.metrics import classification_report

from sklearn.metrics import f1_score, recall_score, confusion_matrix, accuracy_score

# Predict on the test set
y_pred = model_xgb.predict(X_test)


# Generate the Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)


print("Confusion Matrix:")
print(conf_matrix)

print(classification_report(y_test, y_pred))
report_dict = classification_report(y_test, y_pred,output_dict=True)


Confusion Matrix:
[[3858  302  153]
 [ 280 3909  124]
 [ 289  207 3817]]
              precision    recall  f1-score   support

           0       0.87      0.89      0.88      4313
           1       0.88      0.91      0.90      4313
           2       0.93      0.88      0.91      4313

    accuracy                           0.90     12939
   macro avg       0.90      0.90      0.90     12939
weighted avg       0.90      0.90      0.90     12939



In [280]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
df_test= pd.read_csv('GUIDE_Test.csv',nrows=1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df_test.columns = df_test.columns.str.lower()

df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])
# df['timestamp'] = pd.to_datetime(df['timestamp'])
df_test['month'] = df_test['timestamp'].dt.month
df_test['day'] = df_test['timestamp'].dt.day
df_test['hour'] = df_test['timestamp'].dt.hour
df_test['minute'] = df_test['timestamp'].dt.minute

# Drop the original timestamp column
df_test.drop('timestamp', axis=1, inplace=True)

import pandas as pd
import numpy as np
from sklearn.utils import resample

# Assuming your dataframe is called 'df' and the column is 'incidentgrade'

# Get the value counts
value_counts = df_test['incidentgrade'].value_counts()

# Find the minimum count
min_count = value_counts.min()

# Create a list to store the balanced dataframes
balanced_dfs = []

# Undersample each class
for class_value in value_counts.index:
    class_df = df_test[df_test['incidentgrade'] == class_value]

    if len(class_df) > min_count:
        # Undersample
        undersampled_df = resample(class_df,
                                   replace=False,    # sample without replacement
                                   n_samples=min_count,
                                   random_state=42)  # reproducible results
        balanced_dfs.append(undersampled_df)
    else:
        # If this class is already at or below the minimum, keep all samples
        balanced_dfs.append(class_df)

# Combine the balanced dataframes
df_balanced = pd.concat(balanced_dfs)

# Verify the new class distribution
print(df_balanced['incidentgrade'].value_counts())

df_test = df_balanced

column =['orgid', 'incidentid', 'alertid', 'detectorid', 'alerttitle',
       'category', 'incidentgrade', 'entitytype', 'evidencerole', 'deviceid',
       'sha256', 'ipaddress', 'url', 'accountsid', 'accountupn',
       'accountobjectid', 'accountname', 'devicename', 'networkmessageid',
       'registrykey', 'registryvaluename', 'registryvaluedata',
       'applicationid', 'applicationname', 'oauthapplicationid', 'filename',
       'folderpath', 'resourceidname', 'osfamily', 'osversion', 'countrycode',
       'state', 'city', 'month', 'day', 'hour', 'minute']

df_test = df_test[column]

df_test =df_test.astype('category')


categorical_column = df_test.select_dtypes(include='category').columns

le = LabelEncoder()
for col in categorical_column:
    df_test[col] = le.fit_transform(df_test[col])
    
X_test_test = df_test.drop(columns=['incidentgrade']) 
y_test_test = df_test['incidentgrade']

incidentgrade
BenignPositive    206
TruePositive      206
FalsePositive     206
Name: count, dtype: int64


In [281]:
from sklearn.metrics import classification_report

from sklearn.metrics import f1_score, recall_score, confusion_matrix, accuracy_score

# Predict on the test set
y_pred_ = model_xgb.predict(X_test_test)


# Generate the Confusion Matrix
conf_matrix = confusion_matrix(y_test_test, y_pred_)


print("Confusion Matrix:")
print(conf_matrix)

print(classification_report(y_test_test, y_pred_))
report_dict = classification_report(y_test_test, y_pred_,output_dict=True)


Confusion Matrix:
[[133   1  72]
 [109  14  83]
 [ 27   0 179]]
              precision    recall  f1-score   support

           0       0.49      0.65      0.56       206
           1       0.93      0.07      0.13       206
           2       0.54      0.87      0.66       206

    accuracy                           0.53       618
   macro avg       0.65      0.53      0.45       618
weighted avg       0.65      0.53      0.45       618



In [191]:
models = [(
    "Random Forest",
    RandomForestClassifier(),
    (X_train,y_train),
    (X_test,y_test)
),(
    "XGBClassifier",
    XGBClassifier(),
    (X_train,y_train),
    (X_test,y_test)
),(
    "GradientBoostingClassifier",
    GradientBoostingClassifier(),
    (X_train,y_train),
    (X_test,y_test)
)]
# (
#     "Logistic regression",
#     LogisticRegression(),
#     (X_train,y_train),
#     (X_test,y_test)
# )(
#     "KNN",
#     KNeighborsClassifier(),
#     (X_train,y_train),
#     (X_test,y_test)
# ),

In [192]:
reports = []
for model_name,model,train_set,test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]

    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test,y_pred,output_dict=True)
    reports.append(report)

In [193]:
mlflow.set_experiment("null_column_removed_top15")
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

for i ,element in enumerate(models):
    model_name = element[0]
    model = element[1]
    report = reports[i]
    {}

    with mlflow.start_run(run_name=model_name):
        mlflow.log_param("model_name",model_name)

        mlflow.log_metric("accuracy",report["accuracy"])
        mlflow.log_metric("recall_class_0",report['0']['recall'])
        mlflow.log_metric("recall_class_1",report['1']['recall'])
        mlflow.log_metric("recall_class_2",report['2']['recall'])
        mlflow.log_metric('f1_score_macro', report['macro avg']['f1-score'])

        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, model_name)
        else:
            mlflow.sklearn.log_model(model, model_name)    
   

2024/09/12 09:17:29 INFO mlflow.tracking.fluent: Experiment with name 'null_column_removed_top15' does not exist. Creating a new experiment.


2024/09/12 09:17:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/09/12 09:17:34 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/556829533304637448/runs/a0c7978f6f6d4df1bf31a78b61bc9854.
2024/09/12 09:17:34 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/556829533304637448.
c:\Users\New\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [09:17:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2024/09/12 09:17:38 WARNING mlflow.mo

In [94]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost
mlflow.set_experiment("validation")
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.set_tag('description', 'removed_high_cardinality')
    mlflow.log_metrics({
        "accuracy" : report_dict['accuracy'],
        "recall_class_0" : report_dict['0']['recall'],
        "recall_class_1" : report_dict['1']['recall'],
        "recall_class_2" : report_dict['2']['recall'],
        "f1_score_macro" : report_dict['macro avg']['f1-score']
        
    })
    mlflow.sklearn.log_model(model,"Random forest")

2024/09/11 13:38:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/09/11 13:38:55 INFO mlflow.tracking._tracking_service.client: 🏃 View run merciful-finch-597 at: http://127.0.0.1:5000/#/experiments/559247487145313223/runs/853fb685d4af4fbea088fee6bf513e28.
2024/09/11 13:38:55 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/559247487145313223.
